Import the necessary packages

In [1]:
import numpy as np
import csv
from csv import DictReader
import collections
from math import sqrt
import numpy as np
import math
import collections
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
traindata='/content/drive/My Drive/Big Data Analytics/Project 4/training_dataset.csv'
testdata='/content/drive/My Drive/Big Data Analytics/Project 4/test_dataset.csv'

Creating a dictionary with users in the test dataset as keys and the movies they rated + ratings as values:

In [4]:
testuserrat = collections.defaultdict(collections.defaultdict)
with open(testdata, 'r') as file:
  f = list(csv.reader(file))[1:]
for rat in f:
  try:
    testuserrat[rat[0]][rat[1]] = rat[2]
  except:
    pass

In [5]:
users = list(testuserrat.keys())
len(users)

100

Creating a dictionary with users in the training dataset as keys and the movies they rated + ratings as values:

In [6]:
# Dictionary with users as keys and movies: rating as values
trainuserrat = collections.defaultdict(collections.defaultdict)
with open(traindata, 'r') as file:
  fl = list(csv.reader(file))[1:]
for rat in fl:
  try:
      trainuserrat[rat[0]][rat[1]] = rat[2]
  except:
      pass

In [7]:
users = list(trainuserrat.keys())
len(users)

943

Creating a dictionary with movies in the training dataset as keys and the users that rated the movie + the rating they gave as values:

In [8]:
# Dictionary with movies as keys and users + ratings as values
trainmovrat = collections.defaultdict(collections.defaultdict)
with open(traindata, 'r') as file:
  fl = list(csv.reader(file))[1:]
for rat in fl:
    try:
      trainmovrat[rat[1]][rat[0]] = rat[2]
    except:
      pass
  

In [10]:
len(trainmovrat.keys())

1665

Function to define cosine similarity between two movies:

In [9]:
# This function takes two movies as parameters and returns the cosine similarity between the two movies.
# Parameters:
# 1. mov1: first movie
# 2. mov2: second movie
# Returns the cosine similarity between two movies
def cos_sim(mov1, mov2):
  # Calcuating the denominator to plug into the formula:
  # 1. Calculating the square root of the sum of squares
  #    of all the ratings for both movies, and then multiplying these values
  denom = sqrt(sum(map(lambda x:int(x)*int(x),list(trainmovrat[mov1].values())))) * sqrt(sum(map(lambda x:int(x)*int(x),list(trainmovrat[mov2].values()))))
  sum_noom = 0
  # Calculating the numerator:
  # 1. Multiplying ratings for users that have rated both movies
  for k, v in trainmovrat[mov1].items():
    try:
      sum_noom += int(trainmovrat[mov1][k])*int(trainmovrat[mov2][k])
    except:
      pass
  try:
    # Returning the similarity, rounded to two places
    return round(sum_noom/denom,2)
  except:
    return 0

Calculating the mean rating for each user in the training dataset, to use in the formula for adjusted cosine similarity:

In [10]:
mean_ratings = collections.defaultdict()
for k, v in trainuserrat.items():
  try:
    mean_ratings[k] = sum(list(map(lambda x: float(x),list(v.values()))))/len(v.values())
  except:
    pass

Function to calculate the adjusted cosine similarity between two movies:

In [11]:
# This function takes two movies as parameters and returns the adjusted cosine similarity between them.
# Parameters:
# 1. mov1: first movie
# 2. mov2: second movie
# Returns the adjusted cosine similarity between two movies
def adj_cos_sim(mov1, mov2):
  # Calcuating the denominator to plug into the formula:
  # 1. Calculating the square root of the sum of squares
  #    of all the ratings, minus the average rating for each user, for both movies, and then multiplying these values
  adj_rat1 = sum([(float(rating) - mean_ratings[user])**2 for user, rating in trainmovrat[mov1].items()])  
  adj_rat2 = sum([(float(rating) - mean_ratings[user])**2 for user, rating in trainmovrat[mov2].items()])
  denom = sqrt(adj_rat1)*sqrt(adj_rat2)
  sum_noom = 0
  # Calculating the numerator:
  # 1. Multiplying ratings for users that have rated both movies, after the 
  #    average rating for the user has been subtracted from the rating for the movie
  for k, v in trainmovrat[mov1].items():
    try:
      sum_noom += (float(trainmovrat[mov1][k])-mean_ratings[k])*(float(trainmovrat[mov2][k])-mean_ratings[k])
    except:
      pass
  try:
    return round(sum_noom/denom,2)
  except:
    return 0

In [12]:
sim_cos = collections.defaultdict(collections.defaultdict)
sim_adj_cos = collections.defaultdict(collections.defaultdict)
final = {'cosine':sim_cos,'adjusted cosine':sim_adj_cos}

Predicting the rating that a user will give a movie:

In [13]:
# Custom function to predict what rating a user will give a movie, with the 
# option to select the kind of similarity to be used in making predictions
# Parameters:
# 1. user: The user who must rate a movie
# 2. movie: The movie for which the user's rating must be predicted
# 3. similarity: The similarity measure to be used
# 4. t: The threshold for the similarity measure above which a movie will be 
#       used for making predictions
# Returns the predicted rating for a movie by the user

# Dictionary to store function names that will be invoked later
methods = {'cosine':cos_sim,'adjusted cosine':adj_cos_sim}
def predict(user, movie, similarity, t):
  similar = collections.defaultdict(collections.defaultdict)
  for movie2 in trainmovrat.keys():
      if movie != movie2:
        try:
          # Using the appropriate function to calculate the similarity
          x = final[similarity][movie][movie2]
        except:
          x = methods[similarity](movie,movie2)
          final[similarity][movie][movie2] = x
          # Only if the similarity score is greater than the threshold, the 
          # movie will be used for making predictions, later
        if x >= t:
          similar[movie][movie2] = x 
            #final[similarity].update(similar)
        #except:
          #pass
  sim_movs = similar[movie]
  numer = 0
  denom = 0
  # For each similar movie pair, the prediction is calculated:
  #   Numerator: sum of (similarity of movie of similar movie*rating that the user gave the movie)
  #   Denominator: sum of similarities of all similar movies
  for k,v in sim_movs.items():
    try:
      numer+= (float(v)*float(trainuserrat[user][k]))
      denom+= float(v)
    except:
      pass
  try:
    return round(numer/denom,2)
  except:
    return 0

Predicting the rating that user 805 will give movie 41, using the cosine similarity, and a threshold of 0.1:

In [14]:
predict('805','41','cosine',0.1)

3.17

Function to calculate the RMSE:

In [15]:
def RMSE(y_actual, y_predicted):
  rms = sqrt(mean_squared_error(y_actual, y_predicted))
  return round(rms,4)

Function to calculate the RMSE for a certain measure:

In [18]:
# Function to calculate the RMSE for a method by taking all the movies 
# in the test dataset, predicting the rating based on the similarity, 
# and then calculating the RMSE between the actual and predicted ratings
# Parameters:
# 1. similarity: The similarity measure being used to make the predictions
# 2. t: threshold for the similarity measure above which movies will be taken 
#       into account for making predictions
# Returns the RMSE for the predictions when the given similarity and threshold 
# given
def prediction_rmse(similarity,t):
  actual = []
  preds = []
  i = 0
  for k, v in testuserrat.items():
    for mov, rat in v.items():
      try:
        actual.append(int(testuserrat[k][mov]))
        preds.append(predict(k,mov,similarity,t))
      except:
        preds.append(0)
        #pass
  return RMSE(actual, preds)
  #return (actual, preds, fails)


RMSE when the cosine similarity measure is used and the threshold is 0.5:

In [21]:
prediction_rmse('cosine',0.5)

2.5099

RMSE when the cosine similarity measure is used and the threshold is 0.1: (a decrease is observed) 

In [19]:
prediction_rmse('cosine',0.1)

1.0086

RMSE when the adjusted cosine similarity measure is used and the threshold is 0.5:

In [22]:
prediction_rmse('adjusted cosine',0.5)

3.5572

RMSE when the adjusted cosine similarity measure is used and the threshold is 0.1:

In [20]:
prediction_rmse('adjusted cosine',0.1)

0.9135

------------------------------------------------------------------------------------

Teamwork Documentation

Collaborative Item Item Filtering Model – Rushya Puttam

* Loading training, test data into python
* Import CSV Reader
* Importing other necessary packages - 
* Idea on creating nested dictionaries
* On Test & Train Datasets: Creating nested dictionary with users as keys and movie, ratings as values
* On Train Dataset: Creating nested dictionary with movies as keys and users, ratings as values
* On Train Dataset: Creating dictionary with keys as users and values as their average movie ratings
* On Train Dataset: Creating dictionary with users and their average rating
* Defining a function for cosine similarity calculation
* Defining a function for adjusted cosine similarity calculation
* Defining a function to predict unknown ratings
* Retrieving only predicted, actual values of ratings
* Passing into given RMSE function and calling it to get the RMSE on test data